In this notebook, we will build a model that can file a product within the Amazon review dataset under one of the six meta-categories that we extracted from the dataset in a previous notebook (Extracting_Meta_Categories_in_Dataset.ipynb). 
The six meta categories under which the model will file all products are:

Meta-Category 1: Portable Electronics

Meta-Category 2: Connected Home Electronics

Meta-Category 3: Office Supplies

Meta-Category 4: Pet Products

Meta-Category 5: Batteries

Meta-Category 6: Kitchen Storage

We will start by downloading and preprocessing the dataset. We will keep only the categories column, as that is the only useful information to train this model. After that, we will apply TF-IDF to vectorize the text data. Next, we will try to build the model with a Naive Bayes classifier algorithm first, second with a Logistic Regression algorithm. 

In [2]:
# Download the dataset from kagglehub 
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")

print("Path to dataset files:", path)

# Check the files in the folder 

print(os.listdir(path))



Path to dataset files: C:\Users\karel\.cache\kagglehub\datasets\datafiniti\consumer-reviews-of-amazon-products\versions\5
['1429_1.csv', 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv', 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv']


In [3]:
# Load the 3 datasets into pandas dataframes 
# load the first dataset
file_path1 = os.path.join(path, "1429_1.csv")
df1 = pd.read_csv(file_path1)
# Display preliminary information about the dataset
print("Dataset 1 shape:", df1.shape)
print("Columns in the dataset 1:", df1.columns)


# load the second dataset
file_path2 = os.path.join(path, "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")
df2 = pd.read_csv(file_path2)
# Display preliminary information about the dataset
print("Dataset 2 shape:", df2.shape)
print("Columns in the dataset 2:", df2.columns)


# load the third dataset
file_path3 = os.path.join(path, "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
df3 = pd.read_csv(file_path3)
# Display preliminary information about the dataset
print("Dataset 3 shape:", df3.shape)
print("Columns in the dataset 3:", df3.columns)



C:\Users\karel\AppData\Local\Temp\ipykernel_85816\142294653.py:4: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file_path1)


Dataset 1 shape: (34660, 21)
Columns in the dataset 1: Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')
Dataset 2 shape: (5000, 24)
Columns in the dataset 2: Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')
Dataset 3 shape: (28332, 24)
Columns i

In [4]:
# Preprocess the datasets

# For simplicity, we can merge datasets 2 and 3 as they have similar shapes
df_full = pd.concat([df2, df3], ignore_index=True)
# Drop all columns except for 'categories' and 'primaryCategories' from the merged dataset
columns_to_keep = ['name', 'categories', 'primaryCategories']
columns_to_drop1 = [col for col in df_full.columns if col not in columns_to_keep]
df_full.drop(columns=columns_to_drop1, inplace=True)
# Drop all columns except for 'categories' from dataset 1
columns_to_keep2 = ['name', 'categories']
columns_to_drop2 = [col for col in df1.columns if col not in columns_to_keep2]
df1.drop(columns=columns_to_drop2, inplace=True)
# Merge the two datasets on 'categories'
df_merged = pd.merge(df_full, df1, on='categories', how='outer')
# Rename the columns for clarity
df_merged.rename(columns={'categories': 'category', 'primaryCategories': 'primary_category'}, inplace=True)
# Display the final merged dataset
print("Final merged dataset:")
print(df_merged.head())
# Save the final merged dataset to a CSV file and save it in the current working directory
path2 = os.getcwd()
output_file = os.path.join(path2, "merged_dataset.csv")
df_merged.to_csv(output_file, index=False)

Final merged dataset:
                                              name_x  \
0  AmazonBasics AA Performance Alkaline Batteries...   
1  AmazonBasics AA Performance Alkaline Batteries...   
2  AmazonBasics AA Performance Alkaline Batteries...   
3  AmazonBasics AA Performance Alkaline Batteries...   
4  AmazonBasics AA Performance Alkaline Batteries...   

                                            category primary_category name_y  
0  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN  
1  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN  
2  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN  
3  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN  
4  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN  


In the next step, we need to create prediction data to train our model on. The prediction data of the model will be one of our six meta-categories. We will create a new column called 'meta-categories' and label the existing categories in there via [XXXXX decide on smartest method to go about this]

In [ ]:
# Print the unique values in the 'category' and 'primary_category' columns
print("Unique categories count:", df_merged['category'].nunique())
print("Unique categories:", df_merged['category'].unique())
print("Unique primary categories:", df_merged['primary_category'].unique())

# Create a csv file with the unique categories
unique_categories = df_merged['category'].unique()
unique_categories_df = pd.DataFrame(unique_categories, columns=['category'])
# Add a new column 'Meta-Category' with empty values
unique_categories_df['Meta-Category'] = ''
# Save the unique categories to a .xlsx file
output_unique_file = os.path.join(path2, "unique_categories.xlsx")
unique_categories_df.to_excel(output_unique_file, index=False)



Unique categories count: 111
Unique categories: ['AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care'
 'AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care'
 'Accessories,USB Cables,Computers & Accessories,Computer Accessories & Peripherals,Electronics,Cables,Cables & Interconnects'
 'Amazon Device Accessories,Kindle Store,Kindle Touch (4th Generation) Accessories,Kindle E-Reader Accessories,Covers,Kindle Touch (4th Generation) Covers'
 'Amazon Devices & Accessories,Amazon Device Accessories,Power Adapters & Cables,Kindle Store,Kindle E-Reader Accessories,Kindle Paper

We used a combination of manual labelling and analysis by ChatGPT to provide labels in the meta-category column. Now we can use the spreadsheet with categories and meta-categories as training/testing data, where x = categories and y = meta-categories. We will continue with training the model in a new notebook.  

In [17]:
# Load Excel file with unique categories and meta-categories as dataframe
import os
path_xlsx = os.getcwd()

# Use os.path.join for correct path handling
excel_path = os.path.join(path_xlsx, "Spreadsheets", "Categories - Meta Categories - Labeled.xlsx")
unique_categories_labeled_df = pd.read_excel(excel_path)

# Turn the Unique Categories dataframe into a dictionary with 'Meta-Category' as keys and 'category' as values
unique_categories_dict = unique_categories_labeled_df.set_index('category')['Meta-Category'].to_dict()


# Map the 'Meta-Category' to the 'category' in the merged dataset
df_merged['Meta-Category'] = df_merged['category'].map(unique_categories_dict)

# Display the final merged dataset with 'Meta-Category'
print("Final merged dataset with 'Meta-Category':")
print(df_merged.head())




Final merged dataset with 'Meta-Category':
                                              name_x  \
0  AmazonBasics AA Performance Alkaline Batteries...   
1  AmazonBasics AA Performance Alkaline Batteries...   
2  AmazonBasics AA Performance Alkaline Batteries...   
3  AmazonBasics AA Performance Alkaline Batteries...   
4  AmazonBasics AA Performance Alkaline Batteries...   

                                            category primary_category name_y  \
0  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
1  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
2  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
3  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
4  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   

  Meta-Category Meta-Category2  
0     Batteries      Batteries  
1     Batteries      Batteries  
2     Batteries      Batteries  
3     B

In [25]:
# Print the dictionary of unique categories and their meta-categories
'''print("Unique categories and their meta-categories:")
for category, meta_category in unique_categories_dict.items():
    print(f"{category}: {meta_category}")
'''
# Check if the 'Meta-Category' contains any NaN or float values
print("Checking for NaN or float values in 'Meta-Category':")
print(df_merged['Meta-Category'].isnull().any())
# Find the rows with NaN or float values in 'Meta-Category'
nan_rows = df_merged[df_merged['Meta-Category'].isnull() | df_merged['Meta-Category'].apply(lambda x: isinstance(x, float))]
# Display the rows with NaN or float values in 'Meta-Category'
print("All Rows with NaN or float values in 'Meta-Category':")
print(nan_rows[['category', 'Meta-Category']].head(20))

Checking for NaN or float values in 'Meta-Category':
True
All Rows with NaN or float values in 'Meta-Category':
                                               category Meta-Category
19229    Categories,Streaming Media Players,Electronics           NaN
19230    Categories,Streaming Media Players,Electronics           NaN
19231    Categories,Streaming Media Players,Electronics           NaN
19232    Categories,Streaming Media Players,Electronics           NaN
19233    Categories,Streaming Media Players,Electronics           NaN
19234    Categories,Streaming Media Players,Electronics           NaN
19235    Categories,Streaming Media Players,Electronics           NaN
19236    Categories,Streaming Media Players,Electronics           NaN
2886016                      mazon.co.uk,Amazon Devices           NaN
2886017                      mazon.co.uk,Amazon Devices           NaN
2886018                      mazon.co.uk,Amazon Devices           NaN
2886019                      mazon.co.uk,Amazon 

In [29]:
# Delete the rows with 'mazon.co.uk, Amazon Devices' in category
df_merged = df_merged[~df_merged['category'].str.contains('mazon.co.uk, Amazon Devices', na=False)]
# Find the rows with NaN or float values in 'Meta-Category' again
nan_rows = df_merged[df_merged['Meta-Category'].isnull() | df_merged['Meta-Category'].apply(lambda x: isinstance(x, float))]
# Display the number of rows with NaN or float values in 'Meta-Category' after deletion
print("Number of rows with NaN or float values in 'Meta-Category' after deletion:", nan_rows.shape[0])
# Delete all the rows with NaN or float values in 'Meta-Category'
df_merged = df_merged.dropna(subset=['Meta-Category'])
# Display the final merged dataset after cleaning
print("Final merged dataset after cleaning:")
print(df_merged.head())

Number of rows with NaN or float values in 'Meta-Category' after deletion: 27
Final merged dataset after cleaning:
                                              name_x  \
0  AmazonBasics AA Performance Alkaline Batteries...   
1  AmazonBasics AA Performance Alkaline Batteries...   
2  AmazonBasics AA Performance Alkaline Batteries...   
3  AmazonBasics AA Performance Alkaline Batteries...   
4  AmazonBasics AA Performance Alkaline Batteries...   

                                            category primary_category name_y  \
0  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
1  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
2  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
3  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   
4  AA,AAA,Electronics Features,Health,Electronics...  Health & Beauty    NaN   

  Meta-Category Meta-Category2  
0     Batteries      Batteries  
1

In [30]:
# Save the final merged dataset with 'Meta-Category' to a new .csv file
output_merged_file = os.path.join(path2, "merged_dataset_with_meta_category.csv")
df_merged.to_csv(output_merged_file, index=False)